In [2]:
from Game import *

Version:  0.2.20


## Rules of the Game

In [3]:
def initial_state():
    board=Board(3,3)
    board.pieces=['.','X','O']
    return board

def show_state(state):
    print(state)
    
def valid_moves(state,player):  # returns a list of all of the possible moves given a state
    moves=[]
    
    for i in range(9):
        if state[i]==0:
            moves.append(i)
    
    return moves
    
def update_state(state,player,move):
    
    new_state=state
    state[move]=player
    
    return new_state

def win_status(state,player):
    # "win" if the player wins
    # "lose" if the player loses
    # "stalemate" if a tie
    # None if the game continues
    
    # 0 1 2
    # 3 4 5
    # 6 7 8
    
    if state[0]==player and state[1]==player and state[2]==player:
        return "win"
    if state[3]==player and state[4]==player and state[5]==player:
        return "win"
    if state[6]==player and state[7]==player and state[8]==player:
        return "win"
    if state[0]==player and state[3]==player and state[6]==player:
        return "win"
    if state[1]==player and state[4]==player and state[7]==player:
        return "win"
    if state[2]==player and state[5]==player and state[8]==player:
        return "win"
    if state[0]==player and state[4]==player and state[8]==player:
        return "win"
    if state[6]==player and state[4]==player and state[2]==player:
        return "win"
    
    if player==1:
        other_player=2
    else:
        other_player=1
        
        
    if not valid_moves(state,other_player):
        return "stalemate"
    
    
    return None
    

## Agents

In [4]:
def human_move(state,player):
    print("""
     0 1 2
     3 4 5
     6 7 8
    """)
    
    move=int(input("What move?"))
    
    return move

human_agent=Agent(human_move)

In [5]:
def random_move(state,player):
    possible_moves=valid_moves(state,player)
    move=random.choice(possible_moves)
    return move


random_agent=Agent(random_move)

In [6]:
from Game.minimax import *
def minimax_move(state,player):

    values,moves=minimax_values(state,player,display=True)
    return top_choice(moves,values)


minimax_agent=Agent(minimax_move)

In [7]:
def skittles_move(state,player,info):
    S=info.S
    last_action=info.last_action
    last_state=info.last_state
    
    
    # if Ive never seen this state before
    if not state in S:
        actions=valid_moves(state,player)

        S[state]=Table()
        for action in actions:
            S[state][action]=3     
    
    move=weighted_choice(S[state])  # weighted across actions
    
    # what if there are no skittles for a particular state?
    # move is None in that case
    
    if move is None:
        # learn a little bit
        if last_state:
            S[last_state][last_action]=S[last_state][last_action]-1
            if S[last_state][last_action]<0:
                S[last_state][last_action]=0
        
        move=random_move(state,player)
    
    return move

def skittles_after(status,player,info):
    S=info.S
    last_action=info.last_action
    last_state=info.last_state

    if status=='lose':
        # learn a little bit
        S[last_state][last_action]=S[last_state][last_action]-1
        if S[last_state][last_action]<0:
            S[last_state][last_action]=0
        
    


skittles_agent=Agent(skittles_move)
skittles_agent.S=Table()
skittles_agent.post=skittles_after


skittles_agent2=Agent(skittles_move)
skittles_agent2.S=Table()
skittles_agent2.post=skittles_after

In [8]:
g=Game()
g.run(minimax_agent,skittles_agent)

====
Game  1
 .  .  . 
 .  .  . 
 .  .  . 

  Choice Time: 2.5981240272521973 seconds 
Player 1 moves 3
 .  .  . 
 X  .  . 
 .  .  . 

Player 2 moves 5
 .  .  . 
 X  .  O 
 .  .  . 

  Choice Time: 0.16729092597961426 seconds 
Player 1 moves 6
 .  .  . 
 X  .  O 
 X  .  . 

Player 2 moves 8
 .  .  . 
 X  .  O 
 X  .  O 

  Choice Time: 0.004871368408203125 seconds 
Player 1 moves 0
 X  .  . 
 X  .  O 
 X  .  O 

Player  1 won.


[1]

In [9]:
skittles_agent.S

{(0, 0, 0, 1, 0, 0, 0, 0, 0): {0: 3, 1: 3, 2: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3},
 (0, 0, 0, 1, 0, 2, 1, 0, 0): {0: 3, 1: 3, 2: 3, 4: 3, 7: 3, 8: 2}}

In [12]:
g=Game(100)
g.display=False
result=g.run(skittles_agent,skittles_agent2)
g.report()

Total number of games:  100
Winning 64.00 percent
Losing 26.00 percent
Tie 10.00 percent


In [14]:
SaveTable(skittles_agent.S,"ttt player 1 skittles.json")
SaveTable(skittles_agent2.S,"ttt player 2 skittles.json")

In [26]:
number_of_batches=100
wins=[]
losses=[]
ties=[]
skittles_agent.S=Table()
for i in range(number_of_batches):
    N=1000
    g=Game(N)
    g.display=False
    result=g.run(skittles_agent,random_agent)
    wins.append(sum([r==1 for r in result]))
    losses.append(sum([r==2 for r in result]))
    ties.append(sum([r==0 for r in result]))
    print(wins[-1],losses[-1],ties[-1])

590 273 137
599 284 117
612 260 128
582 296 122
620 243 137
597 273 130
600 268 132
633 245 122
613 253 134
647 220 133
640 221 139
640 242 118
655 205 140
649 232 119
677 207 116
660 208 132
662 198 140
669 204 127
677 201 122
693 180 127
675 197 128
692 199 109
729 156 115
701 138 161
692 185 123
714 150 136
711 159 130
723 144 133
724 139 137
727 148 125
725 140 135
729 133 138
732 113 155
743 122 135
734 143 123
755 113 132
737 114 149
753 112 135
766 108 126
769 91 140
772 90 138
774 96 130
761 95 144
782 80 138
784 80 136
773 82 145
793 66 141
761 76 163
784 66 150
795 62 143
804 47 149
800 54 146
813 57 130
783 61 156
808 47 145
804 41 155
804 48 148
798 57 145
809 38 153
811 35 154
830 26 144
817 45 138
819 28 153
797 37 166
813 35 152
820 34 146
811 15 174
805 25 170
818 33 149
835 15 150
818 17 165
829 10 161
817 14 169
832 9 159
825 13 162
828 8 164
821 15 164
813 13 174
814 10 176
819 11 170
826 10 164
834 8 158
809 5 186
851 7 142
830 10 160
837 12 151
831 9 160
823 9 168
